In [4]:
import pandas as pd
from datasets import Dataset, DatasetDict
dir = "../cache_data"
train_data = pd.read_parquet(f'{dir}/train.parquet', engine='pyarrow')
test_data = pd.read_parquet(f'{dir}/test.parquet', engine='pyarrow')

train_ds = Dataset.from_pandas(train_data)
test_ds = Dataset.from_pandas(test_data)

def add_txid(row, i):
    row["transcript_id"] = str(i)
    return row
    
test_ds = test_ds.map(add_txid, with_indices=True)

dataset = DatasetDict(dict(train=train_ds, test=test_ds))

Map:   0%|          | 0/1039 [00:00<?, ? examples/s]

In [17]:
dataset.push_to_hub("diwank/michelleyun-therapydata", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def to_transcript(group):
    records = group.to_dict('records')
    topic = records[0]["topic"]
    transcript_id = int(records[0]["transcript_id"])
    
    data = dict(
        transcript_id=transcript_id,
        topic=topic,
        transcript=[
            dict(
                interlocutor=row["interlocutor"],
                utterance=row["utterance_text"],
                therapist_behavior=row["main_therapist_behaviour"],
                client_talk_type=row["client_talk_type"],
            )
            for row in records
        ]
    )

    return pd.DataFrame([data])

train_data = train_data.groupby("transcript_id").apply(to_transcript)
dataset = Dataset.from_pandas(train_data)
dataset.push_to_hub("diwank/therapydata", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]